In [1]:
from glob import glob

files = sorted(glob('chatgpt-noisy-facebook-translation/*'))
files

['chatgpt-noisy-facebook-translation/facebook-data.jsonl00.splitted.gpt.requested',
 'chatgpt-noisy-facebook-translation/facebook-data.jsonl01.splitted.gpt.requested',
 'chatgpt-noisy-facebook-translation/facebook-data.jsonl02.splitted.gpt.requested',
 'chatgpt-noisy-facebook-translation/facebook-gpt.json',
 'chatgpt-noisy-facebook-translation/facebook-more.json']

In [2]:
import json

facebook = []
for f in files:
    with open(f) as fopen:
        if 'gpt.requested' in f:
            for l in fopen:
                data = json.loads(l)
                facebook.append(data)
        else:
            data = json.load(fopen)
            facebook.extend(data)
            
facebook = [a for a in facebook if a is not None]

In [3]:
len(facebook)

190897

In [4]:
from tqdm import tqdm

filtered = []
for d in tqdm(facebook):
    if 'src' in d:
        left = d['src']['text']
        try:
            en = d['r']['english']
            ms = d['r']['malay']
        except:
            en = None
            ms = None
    else:
        left = d[0]
        success = False
        try:
            r = json.loads(d[1])
            success = True
        except:
            pass
        
        if not success:
            try:
                r = eval(d[1])
                success = True
            except:
                pass
                    
        if success:
            if isinstance(r, tuple):
                en = [r_['english'] for r_ in r]
                ms = [r_['malay'] for r_ in r]
                en = ' '.join(en)
                ms = ' '.join(ms)
            else:
                en = r['english']
                ms = r['malay']
        else:
            en = None
            ms = None
    
    if en or ms:
        filtered.append({
            'left': left,
            'en': en,
            'ms': ms
        })           

100%|███████████████████████████████████████████████████████████████████████████████████| 190897/190897 [00:00<00:00, 1175862.50it/s]


In [5]:
len(filtered), len(facebook)

(141912, 190897)

In [6]:
filtered[0]

{'left': 'Kalau UMNOBN berkuasa, petrol tentu tidak dah subsidi. .. .',
 'en': 'If UMNOBN is in power, there will definitely be no petrol subsidy...',
 'ms': 'Jika UMNOBN berkuasa, pasti tidak akan ada subsidi petrol...'}

In [7]:
def reject(k):
    if 'saya tidak dapat menterjemah teks' in k.lower():
        return
    if 'saya tidak boleh menterjemah kandungan yang tidak sesuai' in k.lower():
        return
    if 'saya tidak boleh menterjemah teks' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai makna' in k.lower():
        return
    if 'teks yang disediakan tidak boleh diterjemahkan' in k.lower():
        return
    if 'teks yang diberikan tidak masuk akal' in k.lower():
        return
    if 'teks yang diberikan nampaknya tidak' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai maksud' in k.lower():
        return
    if 'teks yang diberikan bukan dalam mana-mana bahasa' in k.lower():
        return
    if 'teks ini tidak boleh diterjemahkan' in k.lower():
        return
    if 'saya tidak dapat menterjemah frasa' in k.lower():
        return
    if 'teks yang disediakan bukan' in k.lower():
        return
    if 'teks yang diberikan tidak jelas' in k.lower():
        return
    if 'teks yang diberikan tidak' in k.lower():
        return
    if 'terjemahan teks kepada' in k.lower():
        return
    if 'saya tidak boleh menterjemah bahasa' in k.lower():
        return
    if 'model bahasa AI' in k:
        return
    if 'bahasa melayu standard' in k.lower():
        return
    if 'teks yang diberikan bukan dalam bahasa' in k.lower():
        return
    if 'teks yang anda berikan bukan' in k.lower():
        return
    if 'teks yang disediakan tidak' in k.lower():
        return
    if 'teks yang diberikan bukan' in k.lower():
        return
    if 'saya tidak dapat menterjemah' in k.lower():
        return
    if 'tetapi teks yang diberikan' in k.lower():
        return
    if 'return JSON' in k:
        return
    
    return True

In [8]:
with open('processed-facebook.jsonl', 'w') as fopen:
    for d in filtered:
        try:
            if not reject(d['ms']):
                print(d)
                continue
                # break
            fopen.write(f'{json.dumps(d)}\n')
        except:
            pass

{'left': 'Tbaik raf', 'en': 'Tbaik raf', 'ms': 'Maaf, saya tidak dapat menterjemahkan frasa ini.'}
{'left': 'cirrai engkol anai citaikkvillai ;c etukkiytu .', 'en': 'Sorry, I cannot translate this text.', 'ms': 'Maaf, saya tidak dapat menterjemahkan teks ini.'}
{'left': 'Uwohhhhsggsgg cunnypap', 'en': 'Uwohhhhsggsgg cunnypap', 'ms': 'Maaf, saya tidak dapat menterjemahkan teks ini karena tidak memiliki arti atau bahasa yang jelas.'}
{'left': 'juboo mung', 'en': 'juboooo munggg', 'ms': 'Maaf, saya tidak dapat menterjemahkan frasa ini.'}
{'left': 'TKPA ada di page tak hilang', 'en': "The text, 'Tkpa ada kt page tk hilang', translates to 'The text is not missing on the page' in standard English and standard Malay.", 'ms': "Teks 'Tkpa ada kt page tk hilang' bermaksud 'Teks tidak hilang pada halaman' dalam Bahasa Inggeris standard dan Bahasa Melayu standard."}
{'left': 'Ijoy Nunez Aamiin', 'en': 'Ijoy Nunez Aamiin', 'ms': 'Saya tidak dapat menterjemahkan nama itu ke dalam bahasa Melayu.'}
{'

{'left': 'Mahamada Al Mahadaya Ente', 'en': 'Mahamada Al Mahadaya Ente', 'ms': 'Maaf, saya tidak dapat menterjemahkan frasa ini ke dalam bahasa Inggeris atau bahasa Melayu kerana ia tidak mempunyai makna yang jelas atau tidak mempunyai makna dalam konteks yang diberikan.'}
{'left': 'Xlayan dap', 'en': 'Xlayan dap', 'ms': 'Maaf, saya tidak dapat menterjemahkan frasa ini.'}
{'left': "'Hmd 'fyq ye", 'en': "Hmd 'fyq ye", 'ms': 'Maaf, saya tidak dapat menterjemahkan teks ini.'}
{'left': 'Acyap Bp', 'en': 'Acyap Bp', 'ms': 'Maaf, saya tidak dapat menterjemahkan teks ini ke bahasa Inggeris dan bahasa Melayu kerana ia tidak mempunyai makna yang jelas.'}
{'left': 'Terbai dsai', 'en': 'Terbai dsai', 'ms': 'Maaf, saya tidak dapat menterjemahkan frasa ini.'}
{'left': 'theme song MLM ph guna', 'en': "Text, 'theme song MLM ph guna', translate text to standard English and standard Malay, return JSON structure", 'ms': "Teks, 'lagu tema MLM ph guna', terjemahkan teks ke Bahasa Inggeris standard dan Bah